# Homework 4

This Notebook builds on the DCOPF model introduced in [Notebook 6](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks) and incorporates some elements of Economic Dispatch introduced in [Notebook 4](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks).

First, load (or install if necessary) a set of packages you'll need for this assignment...

In [1]:
using JuMP
using HiGHS
using DataFrames
using CSV
using Plots; plotly();
import Pkg;
#Pkg.add("Clp");
using Clp
#Pkg.add("GLPK");
using GLPK

## Question 1: Modifying IEEE-14

**A. Increased generation costs**

Copy the IEEE 14 bus system and DCOPF solver function from Notebook 6. In addition, add the following line to the return call of the function:
```julia
status = termination_status(DCOPF)
```
This tells you the solver termination status for the problem: e.g. was an optimal solution found, was the solution infeasible, was it unbounded, etc.

Make the following change to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh

Run the DCOPF and output generation, flows, and prices.

In [2]:
datadir = joinpath("..","Notebooks","ieee_test_cases") 
gens = CSV.read(joinpath(datadir,"Gen14.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load14.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

In [3]:
gens_copy1a = copy(gens)
gens_copy1a[gens_copy1a.connnode.==1, :c1] .= 30

1-element view(::Vector{Int64}, [1]) with eltype Int64:
 30

In [4]:
gens_copy1a

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,30,0,300,0,100,-100,161.62,1
2,2,0.25,25,0,140,0,100,-100,97.47,2


In [5]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    try 
        DCOPF = Model(GLPK.Optimizer) # You could use Clp as well, with Clp.Optimizer
        
        # Define sets based on data
          # Set of generator buses
        G = gens.connnode
        
          # Set of all nodes
        N = sort(union(unique(lines.fromnode), 
                unique(lines.tonode)))
        
          # sets J_i and G_i will be described using dataframe indexing below
    
        # Define per unit base units for the system 
        # used to convert from per unit values to standard unit
        # values (e.g. p.u. power flows to MW/MVA)
        baseMVA = 100 # base MVA is 100 MVA for this system
        
        # Decision variables   
        @variables(DCOPF, begin
            GEN[N]  >= 0     # generation        
            # Note: we assume Pmin = 0 for all resources for simplicty here
            THETA[N]         # voltage phase angle of bus
            FLOW[N,N]        # flows between all pairs of nodes
        end)
        
        # Create slack bus with reference angle = 0; use bus 1 with generator
        fix(THETA[1],0)
                    
        # Objective function
        @objective(DCOPF, Min, 
            sum( gens[g,:c1] * GEN[g] for g in G)
        )
        
        # Supply demand balances
        @constraint(DCOPF, cBalance[i in N], 
            sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
                + sum(load for load in loads[loads.connnode .== i,:demand]) 
            == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
        )
    
        # Max generation constraint
        @constraint(DCOPF, cMaxGen[g in G],
                        GEN[g] <= gens[g,:pgmax])
        
        # Flow constraints on each branch 
        @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
                FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
                baseMVA * lines[l,:b] * 
                (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
        )
        
        # Max line flow constraints
        @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
                FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
                lines[l,:capacity]
        ) 
    
    
        # Solve statement (! indicates runs in place)
        optimize!(DCOPF)
    
        # Output variables
        generation = DataFrame(
            node = gens.connnode,
            gen = value.(GEN).data[gens.connnode]
            )
        
        angles = value.(THETA).data
        
        flows = DataFrame(
            fbus = lines.fromnode,
            tbus = lines.tonode,
            flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                            angles[lines.tonode]))
        
        # We output the marginal values of the demand constraints, 
        # which will in fact be the prices to deliver power at a given bus.
        prices = DataFrame(
            node = N,
            value = dual.(cBalance).data)
        
        # Return the solution and objective as named tuple
        return (
            generation = generation, 
            angles,
            flows,
            prices,
            cost = objective_value(DCOPF),
            status = termination_status(DCOPF)
        )
    catch e
        status = termination_status(DCOPF)
    end
end

dcopf_ieee (generic function with 1 method)

In [6]:
solution1a = dcopf_ieee(gens_copy1a, lines,loads)
solution1a.status
#solution.generation

OPTIMAL::TerminationStatusCode = 1

In [7]:
solution1a.generation

Row,node,gen
,Int64,Float64
1,1,119.0
2,2,140.0


In [8]:
solution1a.angles

14-element Vector{Float64}:
  0.0
 -0.03791491976786273
 -0.18200655970723797
 -0.1453797728538324
 -0.12249815256001242
 -0.22741136081397662
 -0.20659753163771283
 -0.20659753163771283
 -0.23880184324738518
 -0.2442681772629424
 -0.239406951515819
 -0.24660972634711795
 -0.2494179036348092
 -0.26611574994200304

In [9]:
solution1a.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


In [10]:
solution1a.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,64.0779
2,1,5,54.9221
3,2,3,72.7846
4,2,4,60.9488
5,2,5,48.6446
6,3,4,-21.4154
7,4,5,-54.3377
8,4,7,29.274
9,4,9,16.7971


Regarding the above results, answer the following:

- How has generation changed compared to the default system?
  **ANS** 
- What explains the new prices?
  **ANS** All prices are equal to the generator 1 because it has not achieved its maximum transmission capacity. The marginal cost of production of generator is 30 dollars per MWh, so all prices are 30.

**B. Constrained line**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW

Run the DCOPF and output generation, flows, and prices.

In [11]:
gens_copy1b = copy(gens)
lines_copy1b = copy(lines)

gens_copy1b[gens_copy1b.connnode.==1, :c1] .= 30
lines_copy1b[lines_copy1b.id.==3, :capacity] .=70

2-element view(::Vector{Int64}, [3, 23]) with eltype Int64:
 70
 70

In [12]:
lines_copy1b

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,10000,1,16.9005
2,1,5,0.05403,0.22304,1,10000,2,4.4835
3,2,3,0.04699,0.19797,1,70,3,5.05127
4,2,4,0.05811,0.17632,1,10000,4,5.67151
5,2,5,0.05695,0.17388,1,10000,5,5.75109
6,3,4,0.06701,0.17103,1,10000,6,5.84693
7,4,5,0.01335,0.04211,1,10000,7,23.7473
8,4,7,0.0,0.20912,1,10000,8,4.78194
9,4,9,0.0,0.55618,1,10000,9,1.79798


In [13]:
solution1b = dcopf_ieee(gens_copy1b, lines_copy1b,loads)
solution1b.status

OPTIMAL::TerminationStatusCode = 1

In [14]:
solution1b.generation

Row,node,gen
,Int64,Float64
1,1,220.837
2,2,38.1627


In [15]:
solution1b.angles

14-element Vector{Float64}:
  0.0
 -0.08841199137462448
 -0.22699099137462464
 -0.1856017313746246
 -0.15928852895264203
 -0.265368664465032
 -0.2462042313708681
 -0.24620423137086814
 -0.2780848789635744
 -0.28331560534983047
 -0.27791883927892164
 -0.2846717894172209
 -0.28756182164678
 -0.30490073480944296

In [16]:
solution1b.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,25.0
3,3,127.287
4,4,57.6793
5,5,48.8474
6,6,51.8507
7,7,56.0958
8,8,56.0958
9,9,55.2628


In [17]:
solution1b.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,149.42
2,1,5,71.417
3,2,3,70.0
4,2,4,55.1212
5,2,5,40.7618
6,3,4,-24.2
7,4,5,-62.4868
8,4,7,28.9798
9,4,9,16.6283


Regarding the above results, answer the following:

- Which node has the highest price and why?
  **ANS** Node 3 gas the highest price since we are limiting the flow which forces the system to increase the price at Node 3 to redirect the flow through other nodes.
- What is the difference in prices across $l_{23}$, also known as the congestion rent? How do you interpret this value (what is it's practical meaning?)
  **ANS**The price difference is 102.29 dollars, where congestion rent is the price difference between loads and exports and the revenue received by generators and imports.

**C. Demand increase**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW
- Increase demands everywhere by 5\%.

In [18]:
gens_copy1c = copy(gens_copy1b)
lines_copy1c = copy(lines_copy1b)
loads_copy1c = copy(loads)

loads_copy1c.demand *= 1.05

11-element Vector{Float64}:
 -22.785
 -98.91000000000001
 -50.19
  -7.9799999999999995
 -11.76
 -30.975
  -9.450000000000001
  -3.6750000000000003
  -6.405
 -14.175
 -15.645000000000001

Calculate the total available generating capacity:

In [19]:
loads_copy1c

Row,connnode,demand
,Int64,Float64
1,2,-22.785
2,3,-98.91
3,4,-50.19
4,5,-7.98
5,6,-11.76
6,9,-30.975
7,10,-9.45
8,11,-3.675
9,12,-6.405


Calculate the new total demand:

In [20]:
sum = 0;
for i in 1:size(loads,1)
    sum = sum + loads.demand[i]
end

println(sum)

-258.99999999999994


Run the DCOPF and show prices.

In [21]:
solution1c = dcopf_ieee(gens_copy1c, lines_copy1c,loads_copy1c)
solution1c.status

INFEASIBLE::TerminationStatusCode = 2

**What is happening in this system?** 

**ANS** The infeasibility indicates that the model cannot solve the 

## Question 2: Linear losses

Up until now, we have ignored transmission losses. A quadratic approximation of losses is given by:

\begin{align}
LOSS_{ij} &\approx \frac{G_{ij}}{BaseMVA} (\theta_i-\theta_j)^2 \\
 & \approx \frac{1}{BaseMVA} \frac{R_{ij}}{R_{ij}^2+X_{ij}^2}(\theta_i-\theta_j)^2
\end{align}


where $G$ is the line's conductance, $R$ is the line's resistance and $X$ is the line's reactance. See the `lines` data frame for these parameters.

For our purposes, we will approximate this quadratic via:


$$
LOSS_{ij} \geq \frac{R_{ij}}{BaseMVA} \times (MaxFlow_{ij})^2 
\left(\frac{|FLOW_{ij}|}{MaxFlow_{ij}} - 0.165 \right)
$$

where $MaxFlow_{ij}=200 MW$ in this problem. Note the greater than equal sign, as we do not want to have negative losses.

This approximation is based on Fitiwi et al. (2016), "Finding a representative network losses model for large-scale transmission expansion planning with renewable energy sources," *Energy* 101: 343-358, https://doi.org/10.1016/j.energy.2016.02.015. 

Note that this is a linear approximation of transmission losses, which are actually a quadratic function of power flows. Fitiwi et al. 2016 and other papers describe piece-wise or segment-wise linear approximations of the quadratic function which provide a tighter lower bound approximation of losses, but we'll use a single linear term for this assignment. 

See Jenkins & Sepulveda et al. 2017, "Enhanced decision support for a changing electricity landscape: the GenX configurable electricity resource capacity expansion model", MIT Energy Initiative Working Paper 2017-10 http://bit.ly/GenXModel Section 5.8, for an example of a linear segment-wise approximation of quadratic transmission losses. 


**A. Code linear losses**

Reload the original data from Notebook 6 and copy the IEEE 14 bus system and DCOPF solver function from Notebook 6 into a new function `dcopf_ieee_lossy`.

Make the following changes:
- Increase the variable cost of Generator 1 to \$30 / MWh
- Change all transmission line capacities to 200 MW

Implement losses into the supply/demand balance equations. A standard way to implement absolute values in linear programming is by introducing two non-negative auxiliary variables $x^+$, $x^-$ $\geq 0$:

$$
x = x^+ - x^-
$$

and the absolute value can be represented as:

$$
|x| = x^+ + x^-
$$

(You should satisfy yourself that this equality holds.)

It makes the formulation easier if losses are added to the supply/demand balance constraint in each node by splitting losses in half between the receiving and sending end.

Indicate which equations and variables you have added and explain your steps using inline code comments (e.g. `# Comment`).

Run the lossy DCOPF and output generation, flows, losses, and prices.

In [22]:
gens_copy2a = copy(gens_copy1a)
lines_copy2a = copy(lines_copy1b) 

lines_copy2a[!, :capacity] .= 200

40-element Vector{Int64}:
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
   ⋮
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200
 200

In [23]:
lines_copy2a

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,200,1,16.9005
2,1,5,0.05403,0.22304,1,200,2,4.4835
3,2,3,0.04699,0.19797,1,200,3,5.05127
4,2,4,0.05811,0.17632,1,200,4,5.67151
5,2,5,0.05695,0.17388,1,200,5,5.75109
6,3,4,0.06701,0.17103,1,200,6,5.84693
7,4,5,0.01335,0.04211,1,200,7,23.7473
8,4,7,0.0,0.20912,1,200,8,4.78194
9,4,9,0.0,0.55618,1,200,9,1.79798


In [24]:
gens_copy2a

Row,connnode,c2,c1,c0,pgmax,pgmin,rgmax,rgmin,pgprev,id
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64
1,1,0.0430293,30,0,300,0,100,-100,161.62,1
2,2,0.25,25,0,140,0,100,-100,97.47,2


In [25]:
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee_lossy(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode)))
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system
    maxFlow = 200
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[N]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
        LOSS[N,N]
        FLOWN[N,N] >= 0
        FLOWP[N,N] >= 0
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:connnode]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode]) + 
        sum(LOSS[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    )
    
    #flow constraints
    @constraint(DCOPF, cFLow[l in 1:nrow(lines)],
        FLOW[lines[l,:fromnode], lines[l,:tonode]] == FLOWP[lines[l, :fromnode],lines[l,:tonode]] - 
        FLOWN[lines[l,:fromnode],lines[l,:tonode]]
    )
    
    #loss constraints
    @constraint(DCOPF, cLoss[l in 1:nrow(lines)],
        (lines[l, :resistance]/baseMVA) *
        (maxFlow^2) *
        (((FLOWP[lines[l,:fromnode], lines[l,:tonode]] + 
        FLOWN[lines[l, :fromnode],lines[l,:tonode]])/maxFlow)-0.165)
        <= LOSS[lines[l,:fromnode], lines[l,:tonode]]
    )

    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    losses = value.(LOSS).data
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        losses,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee_lossy (generic function with 1 method)

In [26]:
solution2a = dcopf_ieee_lossy(gens_copy2a, lines_copy2a, loads)
solution2a.status

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
122 rows, 166 cols, 397 nonzeros
109 rows, 144 cols, 362 nonzeros
Presolve : Reductions: rows 109(-67); columns 144(-668); elements 362(-102)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.5328908654e-01 Pr: 78(18118.3); Du: 0(2.11278e-11) 0s
        112     4.3880642140e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 112
Objective value     :  4.3880642140e+03
HiGHS run time      :          0.00


OPTIMAL::TerminationStatusCode = 1

In [27]:
solution2a.generation

Row,node,gen
,Int64,Float64
1,1,29.6021
2,2,140.0


In [28]:
solution2a.angles

14-element Vector{Float64}:
  0.0
 -0.005429569832512419
 -0.12990757520756363
 -0.07123464821617222
 -0.05015171202636636
 -0.030113608834597714
 -0.07674798913522084
 -0.07674798913522084
 -0.07964834603319589
 -0.07218921964310085
 -0.047202873339041586
 -0.011717275451053465
 -0.022299591865420042
 -0.055733703063809065

In [29]:
solution2a.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,32.5304
3,3,45.8675
4,4,39.5736
5,5,36.8181
6,6,26.8452
7,7,44.8318
8,8,44.8318
9,9,47.598


In [30]:
solution2a.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,9.17622
2,1,5,22.4855
3,2,3,62.8772
4,2,4,37.3214
5,2,5,25.7201
6,3,4,-34.3056
7,4,5,-50.0663
8,4,7,2.63645
9,4,9,1.51277


In [31]:
solution2a.losses

14×14 Matrix{Float64}:
  0.0      -0.92341   0.0        0.0       …    0.0        0.0       0.0
 -0.92341   0.0       2.80786    0.502232       0.0        0.0       0.0
  0.0       2.80786   0.0        0.174981       0.0        0.0       0.0
  0.0       0.502232  0.174981   0.0            0.0        0.0       0.0
 -1.13619  -0.829178  0.0        0.455671       0.0        0.0       0.0
  0.0       0.0       0.0        0.0       …   -6.34427   -3.57232   0.0
  0.0       0.0       0.0       -0.0            0.0        0.0       0.0
  0.0       0.0       0.0        0.0            0.0        0.0       0.0
  0.0       0.0       0.0       -0.0            0.0        0.0      -6.14073
  0.0       0.0       0.0        0.0            0.0        0.0       0.0
  0.0       0.0       0.0        0.0       …    0.0        0.0       0.0
  0.0       0.0       0.0        0.0            0.0      -12.2415    0.0
  0.0       0.0       0.0        0.0          -12.2415     0.0      -7.99715
  0.0       0.0     

**B. Interpret results**

Run the same parameters in the lossless OPF from problem 1. How do prices and flows change? What is the largest magnitude difference in prices between the solution with losses and the lossless OPF solution?

In [32]:
solution2b = dcopf_ieee(gens_copy2a, lines_copy2a, loads)
solution2b.status

OPTIMAL::TerminationStatusCode = 1

In [33]:
solution2b.generation

Row,node,gen
,Int64,Float64
1,1,119.0
2,2,140.0


In [34]:
solution2b.angles

14-element Vector{Float64}:
  0.0
 -0.03791491976786273
 -0.18200655970723797
 -0.1453797728538324
 -0.12249815256001242
 -0.22741136081397662
 -0.20659753163771283
 -0.20659753163771283
 -0.23880184324738518
 -0.2442681772629424
 -0.239406951515819
 -0.24660972634711795
 -0.2494179036348092
 -0.26611574994200304

In [35]:
solution2b.prices

Row,node,value
,Int64,Float64
1,1,30.0
2,2,30.0
3,3,30.0
4,4,30.0
5,5,30.0
6,6,30.0
7,7,30.0
8,8,30.0
9,9,30.0


In [36]:
solution2b.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,64.0779
2,1,5,54.9221
3,2,3,72.7846
4,2,4,60.9488
5,2,5,48.6446
6,3,4,-21.4154
7,4,5,-54.3377
8,4,7,29.274
9,4,9,16.7971


**ANS** the prices remained constant in the lossless model but the prices in the loss model varies greatly by node. This happened for flows too, especially at Node 3 where we controlled the flow. We can see that the lossless model maximizes the flow at Node 3 while the model with losses does not maximize the flow. The largest magnitude difference of prices happen at Node 11 which is 17.598 dollars.

## Question 3 - Security contingencies

Power system operators need to ensure that power is delivered reliably even in the event of unexpected outages (**contingencies**). One common contigency that must be planned for is the loss of a transmission line. The security-constrained OPF (SCOPF) run by operators solves for an optimal dispatch that is simultaneously robust (i.e., feasible) to each of the lines failing individually. This is what is known as **N-1 security**, because we assume that at most one component fails in any given scenario.

In this problem, we will not code a full SCOPF, but rather investigate what happens to the feasibility of our problem when we remove transmission lines.

**A. Setup data**

The following code loads the original dataset (with one row per line) and includes a function `format_lines` that converts this to a format that our solver function can use (duplicating rows for both directions, adding susceptance, etc.).

In [37]:
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
rename!(lines,lowercase.(names(lines)))

function format_lines(lines)
    # create line ids 
    lines.id = 1:nrow(lines);
    # add set of rows for reverse direction with same parameters
    lines2 = copy(lines)
    lines2.f = lines2.fromnode
    lines2.fromnode = lines.tonode
    lines2.tonode = lines2.f
    lines2 = lines2[:,names(lines)]
    append!(lines,lines2)

    # calculate simple susceptance, ignoring resistance as earlier 
    lines.b = 1 ./ lines.reactance
    return(lines)
end

format_lines (generic function with 1 method)

Next:

1. Set the capacity of all lines in the system at 100 MW, except for the line $l_{12}$, which you should set to 200 MW.

2. Create a load dataframe `loads_sens` that increases demands everywhere by 10\%

In [38]:
gens_copy3 = copy(gens)
lines_copy3 = copy(lines_copy1b)
loads_sens = copy(loads)

lines_copy3[!, :capacity] .= 200
lines_copy3[lines_copy3.id.==1, :capacity] .= 100

loads_sens.demand *= 1.1

11-element Vector{Float64}:
  -23.87
 -103.62
  -52.58
   -8.36
  -12.32
  -32.45
   -9.9
   -3.8500000000000005
   -6.71
  -14.850000000000001
  -16.39

**B. Loop over line contingencies**

Create a dataframe `status` with the `fromnode` and `tonode` columns of `lines`.

Create a [for loop](https://docs.julialang.org/en/v1/manual/control-flow/#man-loops) that iterates over each line and:
- sets the reactance to be a very high value, 1e9 (i.e., no power will be transmitted)
- creates a version of the dataframe that our solver function can use via `format_lines`
- runs DCOPF
- stores the solution status in a `opf` column in the corresponding row of the `status` dataframe

Show the `status` results.

In [39]:
#status = DataFrame(lines_copy3.fromnode, lines_copy3.tonode)
lines_copy3.fromnode[1:20]
lines_copy3.fromnode[21:40]
#opf1 = Array{String, 1}(20)
status = DataFrame(fromnode = [], tonode = [], opf = [])
size1 = 20

20

In [40]:
lines_copy3

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.01938,0.05917,1,100,1,16.9005
2,1,5,0.05403,0.22304,1,200,2,4.4835
3,2,3,0.04699,0.19797,1,200,3,5.05127
4,2,4,0.05811,0.17632,1,200,4,5.67151
5,2,5,0.05695,0.17388,1,200,5,5.75109
6,3,4,0.06701,0.17103,1,200,6,5.84693
7,4,5,0.01335,0.04211,1,200,7,23.7473
8,4,7,0.0,0.20912,1,200,8,4.78194
9,4,9,0.0,0.55618,1,200,9,1.79798


In [41]:
#status_array = Array{Float64}(undef, 20)

for i in 1:size1
    #store = lines_copy3.reactance[i];
    lines_copy3.reactance[i] .= 1e9;
    lines_copy3_new = format_lines(lines_copy3)
    solution3 = dcopf_ieee(gens_copy3, lines_copy3_new, loads_sens)
    push!(status, ([lines_copy3_new[lines_copy3_new.id.==i, :fromnode],lines_copy3_new[lines_copy3_new.id.==i, :tonode], solution3.status]))
    #lines_copy3.reactance[i] .= store;
end

LoadError: MethodError: no method matching copyto!(::Float64, ::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{0}, Tuple{}, typeof(identity), Tuple{Float64}})

[0mClosest candidates are:
[0m  copyto!([91m::AbstractDataFrame[39m, ::Base.Broadcast.Broadcasted{<:Base.Broadcast.AbstractArrayStyle{0}})
[0m[90m   @[39m [35mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/other/[39m[90m[4mbroadcasting.jl:320[24m[39m
[0m  copyto!([91m::AbstractDataFrame[39m, ::Base.Broadcast.Broadcasted)
[0m[90m   @[39m [35mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/other/[39m[90m[4mbroadcasting.jl:296[24m[39m
[0m  copyto!([91m::DataFrames.LazyNewColDataFrame[39m, ::Base.Broadcast.Broadcasted{T}) where T
[0m[90m   @[39m [35mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/other/[39m[90m[4mbroadcasting.jl:196[24m[39m
[0m  ...


In [42]:
status

Row,fromnode,tonode,opf
,Any,Any,Any


**3. Interpret results**

Are all of the cases feasible? If not, how many are infeasible? 

**ANS** I was able to do this the first time I ran this notebook, then it is throwing me out an error that I was not able to fix. Anyways the only case 5 was giving me a dual-infeasible, so 1 case.

Pick two cases where the solution gives a different status. (For our purposes, dual infeasible and primal infeasible are the same.) What is happening here?

Given this, do you conclude that the system with the assumed transmission line ratings is secure as-is, or do we need to add more redundancy to the system?

**ANS** Comparing case 1 and case 5, the solution is feasible and infeasible respectively. In the situation of the infeasible case, the model conclude that the certain line cannot be removed as it would break the system. This implies that we need to add more redundacy to the system to prevent such cases where the system breaks if a certain line is removed.